# DEMO: GALAXY CLUSTERING SPECTROSCOPIC MAGNIFICATION BIAS

This notebook serves as a verification for the calculation of the density-magnification and magnification-magnifiation  realspace correlation functions ($\xi_{\ell}^{g\mu}$ and $\xi_{\ell}^{\mu\mu}$ respectively) when the spectroscopic magnification bias $s(z)$ is taken into account. 

The relevant equations as spelled out in the overleaf document (**insert link here?**) have been implemented in the `Spectro` module of CLOE. Here, we plot them against benchmark data that has been generated by the external code `COFFE`.

In [ ]:
from matplotlib import logging
logging.getLogger('matplotlib.font_manager').disabled = True

# General python imports
import numpy as np
import matplotlib.pyplot as plt
import time 
import os, sys

#Setting directories

likelihood_path = os.path.realpath(os.path.join(os.getcwd(),'..'))
sys.path.insert(0, likelihood_path)
print('Setting as working directory: ', likelihood_path)

from cloe.cobaya_interface import EuclidLikelihood

# Matplotlib params set-up

%matplotlib inline
plt.rc('xtick',labelsize=16)
plt.rc('ytick',labelsize=16)
plt.rc('font',size=25)
plt.rc('axes', titlesize=26)
plt.rc('axes', labelsize=25)
plt.rc('lines', linewidth=2)
plt.rc('lines', markersize=6)
plt.rc('legend', fontsize=20)
plt.rc('mathtext', fontset='stix')
plt.rc('font', family='STIXGeneral')

## Setup Cobaya `model` and calculate observables for single point

In [ ]:
# Setup info dictionary as required by Cobaya.
info = {
    'params': {
            'ombh2': 0.049*0.67*0.67, #Omega density of baryons times the reduced Hubble parameter squared
            'omch2': 0.27*0.67*0.67, #Omega density of cold dark matter times the reduced Hubble parameter squared
            'H0': 67.0, #Hubble parameter evaluated today (z=0) in km/s/Mpc
            'tau': 0.053301, #optical depth
            'mnu': 0.0, #  sum of the mass of neutrinos in eV
            'nnu': 3.046, #N_eff of relativistic species 
            'As': 2.114e-09, #Amplitude of the primordial scalar power spectrum
            'ns': 0.96, # primordial power spectrum tilt (sampled with an uniform prior)
            'w': -1.0, #Dark energy fluid model
            'wa': 0.0, #Dark energy fluid model
            'omk': 0.0, #curvature density
            'omegam': None, #DERIVED parameter: Omega matter density
            'omegab': None, #DERIVED parameter: Omega baryon density
            'omeganu': None, #DERIVED parameter: Omega neutrino density
            'omnuh2': 0.0, #DERIVED parameter: Omega neutrino density times de reduced Hubble parameter squared
            'omegac': None, #DERIVED parameter: Omega cold dark matter density
        'N_eff': None},
    #'theory': Cobaya's protected key of the input dictionary.
    'theory': {'camb': 
               {'stop_at_error': True, 
                'extra_args':{'num_massive_neutrinos': 0,
                              'dark_energy_model': 'ppf'}}},
    #'sampler': Cobaya's protected key of the input dictionary. 
    'sampler': {'evaluate': None},  
    'output': 'chains/my_euclid_experiment',
    #'debug': how much information you want Cobaya to print? If debug: True, it prints every single detail
    # that is going on internally in Cobaya
    'debug': False,
    #'timing':  if timing: True, Cobaya returns how much time it took it to make a computation of the posterior
    # and how much time take each of the modules to perform their tasks
    'timing': True,
    #'force': 'force': True, Cobaya forces deleting the previous output files, if found, with the same name
    'force': True,
    }

In [ ]:
#'likelihood': Cobaya's protected key of the input dictionary.
info['likelihood'] = {'Euclid': 
                     {'external': EuclidLikelihood, # Likelihood Class to be read as external
                     'observables_selection': {
                         'WL': {'WL': False, 'GCphot': False, 'GCspectro': False},
                         'GCphot': {'GCphot': False, 'GCspectro': False},
                         'GCspectro': {'GCspectro': True},
                         'CG': {'CG': False},
                         'add_phot_RSD': False,
                         'matrix_transform_phot': False,
                     },
                     'plot_observables_selection': False,  
                     'matrix_transform_phot' : False, # 'BNT' or 'BNT-test,
                     'NL_flag_phot_matter': 0,
                     'NL_flag_spectro': 0,
                     'NL_flag_phot_baryon': 0,
                     'NL_flag_phot_bias': 0,
                     'IA_flag': 0,
                     'IR_resum': 'DST',
                     'Baryon_redshift_model': False,
                     'use_magnification_bias_spectro': 0,
                     'solver': 'camb',
                     'use_Weyl': False,
                     # k values for extrapolation of the matter power spectrum and size k-array
                     'k_max_extrap': 500.0,
                     'k_min_extrap': 1E-5,   
                     'k_samp': 1000,
                     # z limit values and size z-array
                     'z_min': 0.0,
                     'z_max': 4.0,
                     'z_samp': 100,
                     # Use MG gamma
                     'use_gamma_MG': False,
                     # Use redshift-dependent purity for GCspectro or not
                     'f_out_z_dep': False,
                     # Add spectroscopic redshift errors
                     'GCsp_z_err' : False,
                     'observables_specifications':{
                         'GCspectro':{
                                   'statistics': 'multipole_correlation_function',
                                   'multipole_correlation_function':{
                                        'bins':{
                                          'n1': {
                                            'n1':{
                                              'multipoles':{
                                                0:{
                                                  'r_range': [[40, 385]]},
                                                2: {
                                                  'r_range': [[40, 385]]},
                                                4:{
                                                  'r_range': [[40, 385]]}
                                                }
                                              }
                                            },
                                          'n2': {
                                            'n2':{
                                              'multipoles':{
                                                0:{
                                                  'r_range': [[40, 385]]},
                                                2:{
                                                  'r_range': [[40, 385]]},
                                                4:{
                                                  'r_range': [[40, 385]]}
                                                }
                                              }
                                            },
                                          'n3': {
                                            'n3': {
                                              'multipoles':{
                                                0:{
                                                  'r_range': [[40, 385]]},
                                                2:{
                                                  'r_range': [[40, 385]]},
                                                4:{
                                                  'r_range': [[40, 385]]}
                                                }
                                              }
                                            },
                                          'n4': {
                                            'n4':{
                                              'multipoles':{
                                                0:{
                                                  'r_range': [[40, 385]]},
                                                2:{
                                                  'r_range': [[40, 385]]},
                                                4:{
                                                  'r_range': [[40, 385]]}}}}}}}},
                     #'data': This give specifications for the paths of the input data files
                     'data': { 
                        #'sample' specifies the first folder below the main data folder
                        'sample': 'ExternalBenchmark',
                        #'spectro' and 'photo' specify paths to data files.
                        'spectro': {
                            # GC Spectro root name should contain z{:s} string
                            # to enable iteration over bins
                            'root': 'cov_coffe_corr_func_z{:s}.fits',
                            'redshifts': ["1.", "1.2", "1.4", "1.65"],
                            'edges': [0.9, 1.1, 1.3, 1.5, 1.8],
                            'scale_cuts_fourier': 'GCspectro-ConfigurationSpace.yaml',
                            'root_mixing_matrix': 'mm_FS230degCircle_m3_nosm_obsz_z0.9-1.1.fits',
                            'Fourier': False},
                        'photo': {
                            'redshifts': [0.2095, 0.489, 0.619, 0.7335, 0.8445, 0.9595, 1.087, 1.2395, 1.45, 2.038],
                            'ndens_GC': 'niTab-EP10-RB00.dat',
                            'ndens_WL': 'niTab-EP10-RB00.dat',
                            'luminosity_ratio': 'luminosity_ratio.dat',
                            # Photometric root names should contain z{:s} string
                            # to specify IA model
                            'root_GC': 'Cls_{:s}_PosPos.dat',
                            'root_WL': 'Cls_{:s}_ShearShear.dat',
                            'root_XC': 'Cls_{:s}_PosShear.dat',
                            'IA_model': 'zNLA',
                            # Photometric covariances root names should contain z{:s} string
                            # to specify how the covariance was calculated
                            'cov_GC': 'CovMat-PosPos-{:s}-20Bins.npy',
                            'cov_WL': 'CovMat-ShearShear-{:s}-20Bins.npy',
                            'cov_3x2pt': 'CovMat-3x2pt-{:s}-20Bins.npy',
                            'cov_model': 'Gauss',  # or 'BNT-Gauss' if BNT selected above
                            'Fourier': False
                        }},
                      'params': {
                                # Spectroscopic bias parameters
                                'b1_spectro_bin1': 1.441,
                                'b1_spectro_bin2': 1.643,
                                'b1_spectro_bin3': 1.862,
                                'b1_spectro_bin4': 2.078,
                                #Spectroscopic magnification bias
                                'magnification_bias_spectro_bin1': 0.79,
                                'magnification_bias_spectro_bin2': 0.87,
                                'magnification_bias_spectro_bin3': 0.96,
                                'magnification_bias_spectro_bin4': 0.98,
                      },
                    }}

In [ ]:
# First: import model wrapper of Cobaya 
from cobaya.model import get_model

t1 = time.time()

# Create an instance of the `model` wrapper called model
model = get_model(info)
print('Time for initialization of the likelihood: ', time.time()-t1)

logposterior = model.logposterior({})
print(logposterior)

In [ ]:
# Create an instance of the class EuclidLikelihood
like = EuclidLikelihood()

# Initialize default parameters for redshift, k-array, fiducial cosmology...
like.initialize()

# Get the cosmo_dictionary where all the cosmology + theory parameters are saved
like.passing_requirements(model, info, **model.provider.params)

# Update the cosmology dictionary with interpolators + basic quantities such as
# P_gg, P_delta...
like.cosmo.update_cosmo_dic(like.cosmo.cosmo_dic['z_win'], 0.05)


## Read COFFE data files and plot GCspectro mag bias for verification 

In [ ]:
import matplotlib
matplotlib.rcParams['text.usetex'] = False
from cloe.spectroscopic_survey import spectro

%matplotlib inline

# Define array of separation r
sep = np.arange(40.0, 390.0, 5.0)
ks = np.logspace(5e-5, 50, 2048)

zs = info['likelihood']['Euclid']['data']['spectro']['redshifts']
start_arr = [0,3,6,9]

#Read COFFE data
out_name_denslens = '../data/ExternalBenchmark/Spectroscopic/data/coffe-data/signal_denslens_2.dat'
file_denslens = np.loadtxt(out_name_denslens)
signal_denslens = file_denslens[:,3]

out_name_lenslens = '../data/ExternalBenchmark/Spectroscopic/data/coffe-data/signal_lenslens_2.dat'
file_lenslens = np.loadtxt(out_name_lenslens)
signal_lenslens = file_lenslens[:,3]

for i, z_mean in enumerate(zs):
    fig, ax = plt.subplots(1,2, figsize=(25,10))
    
    spectro_obj = spectro.Spectro(like.cosmo.cosmo_dic, float(z_mean))
    
    t1 = time.time()
    #Calculate density-magnification bias correlation function
    xi_g_mu_mono = spectro_obj.multipole_correlation_function_dens_mag(sep, float(z_mean), 0)
    xi_g_mu_quad = spectro_obj.multipole_correlation_function_dens_mag(sep, float(z_mean), 2)
    xi_g_mu_exad = spectro_obj.multipole_correlation_function_dens_mag(sep, float(z_mean), 4)
    print('Done calculating dens-mag bias. Time taken =', time.time()-t1)
    
    #Calculate magnification-magnification bias correlation function
    t1 = time.time()
    xi_mu_mu_mono = spectro_obj.multipole_correlation_function_mag_mag(sep, float(z_mean), 0)
    xi_mu_mu_quad = spectro_obj.multipole_correlation_function_mag_mag(sep, float(z_mean), 2)
    xi_mu_mu_exad = spectro_obj.multipole_correlation_function_mag_mag(sep, float(z_mean), 4)
    print('Done calculating mag-mag bias. Time taken = ', time.time()-t1)

    #Plot comparison with COFFE data
    start = start_arr[i]

    # Plot xi_g_mu
    ax[0].plot(sep, signal_denslens[start*len(sep):(start+1)*len(sep)], ls = '-', lw = '2', color='C0', label = r'$\ell = 0$')
    ax[0].plot(sep, xi_g_mu_mono, ls = '--', lw = '2', color='k')
    
    ax[0].plot(sep, signal_denslens[(start+1)*len(sep):(start+2)*len(sep)], ls = '-', lw = '2', color='C1', label = r'$\ell = 2$')
    ax[0].plot(sep, xi_g_mu_quad, ls = '--', lw = '2', color='k')
    
    ax[0].plot(sep, signal_denslens[(start+2)*len(sep):(start+3)*len(sep)], ls = '-', lw = '2', color='C2', label = r'$\ell = 4$')
    ax[0].plot(sep, xi_g_mu_exad, ls = '--', lw = '2', color='k')
    
    ax[0].set_xlabel(r'$r\,[\mathrm{Mpc}]$', fontsize = 40)
    ax[0].text(320.0, 0.9*max(xi_g_mu_quad), r'$\bar{z} = '+str(z_mean)+'$', fontsize=32)
    ax[0].set_ylabel(r'$\xi^{\delta\mu}_\ell(r)[\mathrm{Mpc}]$', fontsize = 40)

    # Plot xi_mu_mu
    ax[1].plot(sep, signal_lenslens[start*len(sep):(start+1)*len(sep)], ls = '-', lw = '2', color='grey', label = r'COFFE')
    ax[1].plot(sep, xi_mu_mu_mono, ls = '--', lw = '2', color='k', label = r'This notebook')
    
    ax[1].plot(sep, signal_lenslens[start*len(sep):(start+1)*len(sep)], ls = '-', lw = '2', color='C0', label = r'$\ell = 0$')
    ax[1].plot(sep, xi_mu_mu_mono, ls = '--', lw = '2', color='k')
    
    ax[1].plot(sep, signal_lenslens[(start+1)*len(sep):(start+2)*len(sep)], ls = '-', lw = '2', color='C1', label = r'$\ell = 2$')
    ax[1].plot(sep, xi_mu_mu_quad, ls = '--', lw = '2', color='k')
    
    ax[1].plot(sep, signal_lenslens[(start+2)*len(sep):(start+3)*len(sep)], ls = '-', lw = '2', color='C2', label = r'$\ell = 4$')
    ax[1].plot(sep, xi_mu_mu_exad, ls = '--', lw = '2', color='k')
    
    ax[1].legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize = 24, frameon=True, ncol = 1)
    ax[1].set_xlabel(r'$r\,[\mathrm{Mpc}]$', fontsize = 40)
    ax[1].text(320.0, 0.9*max(xi_mu_mu_quad), r'$\bar{z} = '+str(z_mean)+'$', fontsize=32)
    ax[1].set_ylabel(r'$\xi^{\mu\mu}_\ell(r)\,[\mathrm{Mpc}]$', fontsize = 40)
    
    fig.show()


In [ ]:
#Read COFFE data
out_name_spectro = '../data/ExternalBenchmark/Spectroscopic/data/coffe-data/2pcf_coffe.dat'
file_spectro = np.loadtxt(out_name_spectro,dtype='float')
signal_mag = file_spectro[:,3]

zs=['1']
for i, z_mean in enumerate(zs):
    fig, ax = plt.subplots(1,1, figsize=(15,10))
    
    spectro_obj = spectro.Spectro(like.cosmo.cosmo_dic, float(z_mean))
    
    #Calculate density-magnification bias correlation function
    xi_mono = spectro_obj.multipole_correlation_function(sep, float(z_mean), 0, k_min=0.0001,k_max=100)
    xi_quad = spectro_obj.multipole_correlation_function(sep, float(z_mean), 2, k_min=0.0001,k_max=100)
    xi_exad = spectro_obj.multipole_correlation_function(sep, float(z_mean), 4, k_min=0.0001,k_max=100)
    
    #Plot comparison with COFFE data
    start = start_arr[i]

    # Plot xi
    ax.plot(sep, sep**2*(signal_mag[start*len(sep):(start+1)*len(sep)]),
            ls = '-', lw = '2', color='C0', label = r'$\ell = 0$')
    ax.plot(sep, sep**2*(xi_mono[0]), ls = '--', lw = '2', color='k')
    
    ax.plot(sep, sep**2*(signal_mag[(start+1)*len(sep):(start+2)*len(sep)]),
            ls = '-', lw = '2', color='C1', label = r'$\ell = 2$')
    ax.plot(sep, sep**2*(xi_quad[0]),ls = '--', lw = '2', color='k')
    
    ax.plot(sep, sep**2*(signal_mag[(start+2)*len(sep):(start+3)*len(sep)]),
            ls = '-', lw = '2', color='C2', label = r'$\ell = 4$')
    ax.plot(sep, sep**2*xi_exad[0], ls = '--', lw = '2', color='k', label = r'This notebook')
    
    ax.set_xlabel(r'$r\,[\mathrm{Mpc}]$', fontsize = 40)
    ax.set_ylabel(r'$r^2\xi(r)[\mathrm{Mpc}^2]$ (w/ mag bias)', fontsize = 30)
    ax.text(250.0, 65, r'$\bar{z} = '+str(z_mean)+'$', fontsize=32)
    ax.legend()

    
    fig.show()

In [ ]:

for i, z_mean in enumerate(zs):
    fig, ax = plt.subplots(1,1, figsize=(15,10))
    
    spectro_obj = spectro.Spectro(like.cosmo.cosmo_dic, float(z_mean))
    
    #Calculate density-magnification bias correlation function
    xi_mono = spectro_obj.multipole_correlation_function(sep, float(z_mean), 0, k_min=0.0001,k_max=100)
    xi_quad = spectro_obj.multipole_correlation_function(sep, float(z_mean), 2, k_min=0.0001,k_max=100)
    xi_exad = spectro_obj.multipole_correlation_function(sep, float(z_mean), 4, k_min=0.0001,k_max=100)
    
    #Plot comparison with COFFE data
    start = start_arr[i]

    coffe_mono = signal_mag[start*len(sep):(start+1)*len(sep)]+2*signal_denslens[start*len(sep):(start+1)*len(sep)]+signal_lenslens[start*len(sep):(start+1)*len(sep)]

    coffe_quad = signal_mag[(start+1)*len(sep):(start+2)*len(sep)]+2*signal_denslens[(start+1)*len(sep):(start+2)*len(sep)]+signal_lenslens[(start+1)*len(sep):(start+2)*len(sep)]

    coffe_exad = signal_mag[(start+2)*len(sep):(start+3)*len(sep)]+2*signal_denslens[(start+2)*len(sep):(start+3)*len(sep)]+signal_lenslens[(start+2)*len(sep):(start+3)*len(sep)]
    
    # Plot xi
    ax.plot(sep, 100*(coffe_mono-xi_mono[0])/((coffe_mono+xi_mono[0])/2), 
            ls = '-', lw = '2', color='C0', label = r'$\ell = 0$')
    
    ax.plot(sep, 100*(coffe_quad-xi_quad[0])/((coffe_quad+xi_quad[0])/2), 
            ls = '-', lw = '2', color='C1', label = r'$\ell = 2$')
    
    ax.plot(sep, 100*(coffe_exad-xi_exad[0])/((coffe_exad+xi_exad[0])/2), 
            ls = '-', lw = '2', color='C2', label = r'$\ell = 4$')
    
    ax.set_xlabel(r'$r\,[\mathrm{Mpc}]$', fontsize = 40)
    ax.set_ylabel('% rel. diff.', fontsize = 30)
    ax.hlines(0.0,xmin=30,xmax=400,color='gray',linestyles='dashed')
    # ax.set_yscale('log')
    # ax.set_ylim([-10,10])
    ax.text(0.9,0.1, r'$\bar{z} = '+str(z_mean)+'$', fontsize=32, horizontalalignment='center', verticalalignment='center', transform = ax.transAxes)
    ax.legend()

    
    fig.show()